In [ ]:
YOUR_NAME = 'Erwin' #change to your name
EXPERIMENT_NAME = 'Scikit Learn CAIP Deploy V1.1 %s' % (YOUR_NAME) # change to your experiment name 

In [ ]:
import kfp.components as comp

mlengine_deploy_op = comp.load_component_from_url(
    'https://raw.githubusercontent.com/kubeflow/pipelines/17a104eac09e29bd39c0edc933f7fc53a5f98ef7/components/gcp/ml_engine/deploy/component.yaml')
help(mlengine_deploy_op)

In [ ]:
import datetime as datetime

# Required Parameters
PROJECT_ID = 'kfp-primer-workshop'

# Optional Parameters
EXPERIMENT_NAME = EXPERIMENT_NAME
TRAINED_MODEL_PATH = 'gs://kfp-scikit/model/output/model/'
VERSION = "Marketing_V_%s_%s" % (datetime.datetime.now(), YOUR_NAME)

In [ ]:
import kfp.dsl as dsl
import kfp.gcp as gcp
import json
@dsl.pipeline(
    name='CAIP deploy pipeline',
    description='CAIP deploy pipeline'
)
def pipeline(
    model_uri = 'gs://kfp-scikit/model/output/model/',
    project_id = PROJECT_ID,
    model_id = 'kfp_sample_model',
    version_id = VERSION,
    runtime_version = '1.14',
    python_version = '3.5',
    version = '',
    replace_existing_version = 'True',
    set_default = 'True',
    wait_interval = '30'):
    task = mlengine_deploy_op(
        model_uri=model_uri, 
        project_id=project_id, 
        model_id=model_id, 
        version_id=version_id, 
        runtime_version=runtime_version, 
        python_version=python_version,
        version=version, 
        replace_existing_version=replace_existing_version, 
        set_default=set_default, 
        wait_interval=wait_interval).apply(gcp.use_gcp_secret('user-gcp-sa'))

In [ ]:
pipeline_func = pipeline
pipeline_filename = pipeline_func.__name__ + '.zip'
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [ ]:
#Specify pipeline argument values
arguments = {}

#Get or create an experiment and submit a pipeline run
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

#Submit a pipeline run
run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)

In [ ]:
{
"instances": [[45,"admin.","divorced","secondary",2353,true,false,"cellular",5,"mar",182,1,107,2,"other"]
]
}